In [1]:
!wget -O Robust04.rar https://dl.dropboxusercontent.com/s/5qwq3gn6rto98sd/Robust04%2Bpos%2Btf%2Bcf.rar?dl=0
!unrar e -o+ Robust04.rar

--2019-11-19 13:29:12--  https://dl.dropboxusercontent.com/s/5qwq3gn6rto98sd/Robust04%2Bpos%2Btf%2Bcf.rar?dl=0
Resolving dl.dropboxusercontent.com (dl.dropboxusercontent.com)... 162.125.65.6, 2620:100:6021:6::a27d:4106
Connecting to dl.dropboxusercontent.com (dl.dropboxusercontent.com)|162.125.65.6|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 460221412 (439M) [application/rar]
Saving to: ‘Robust04.rar’

Robust04.rar        100%[===================>] 438.90M  46.8MB/s    in 16s     

2019-11-19 13:29:29 (26.8 MB/s) - ‘Robust04.rar’ saved [460221412/460221412]


UNRAR 5.50 freeware      Copyright (c) 1993-2017 Alexander Roshal


Extracting from Robust04.rar

Extracting  terms.csv                                                      0%  1%  2%  3%  4%  5%  6%  7%  8%  9% 10% 11% 12% 13% 14% 15% 16% 17% 18% 19% 20% 21% 22% 23% 24% 25% 26% 27% 28%

In [0]:
from datetime import datetime

In [3]:
!pip install duckdb
import duckdb

class DuckDB(object):
  """
  Class that houses all the DuckDB functionalities.

  Attributes:
    c         = [Cursor] database cursor of DuckDB
    C         = [int] number of indexed terms
    N         = [int] number of indexed documents
    avgdl     = [float] average number of terms per document
    len_query = [int] number of terms in current search query
  """
  def __init__(self,
               database=':memory:',
               dict='dict.csv',
               docs='docs.csv',
               terms='terms.csv'):
    """
    Initializes DuckDB database with index and statistics.

    Args:
      database = [str] database path
      dict     = [str] filename for dictionary CSV
      docs     = [str] filename for documents CSV
      terms    = [str] filename for terms CSV
    """
    con = duckdb.connect(database)
    self.c = con.cursor()

    # copy dictionary CSV into DuckDB database
    self.c.execute("CREATE TABLE dict(termid INTEGER "
                                    ",term   VARCHAR "
                                    ",df     INTEGER "
                                    ",cf     INTEGER)")
    self.c.execute("COPY dict "
                   "FROM '" + dict + "' "
                   "WITH DELIMITER '|'")
    
    # copy documents CSV into DuckDB database
    self.c.execute("CREATE TABLE docs(name  VARCHAR "
                                    ",docid INTEGER "
                                    ",len   INTEGER "
                                    ",temp  INTEGER)")
    self.c.execute("COPY docs "
                   "FROM '" + docs + "' "
                   "WITH DELIMITER '|'")
    
    # copy terms CSV into DuckDB database
    self.c.execute("CREATE TABLE terms(termid INTEGER "
                                     ",docid  INTEGER "
                                     ",pos    INTEGER "
                                     ",tf     INTEGER)")
    self.c.execute("COPY terms "
                   "FROM '" + terms + "' "
                   "WITH DELIMITER '|'")
    
    # compute standard index statistics
    self.C = self._C()
    self.N = self._N()
    self.avgdl = self._avgdl()
    self.len_query = 0
    
  def make_query(self, *args: str):
    """
    Makes query table in DuckDB database filled with query terms.
    
    Args:
      args = [[str]] concatenation of strings to be made into a query
    """
    query = "('" + args[0] + "')"
    for arg in args[1:]:
        query += ", ('" + arg + "')"
    
    self.c.execute("DROP TABLE IF EXISTS query")
    self.c.execute("CREATE TABLE query(term VARCHAR)")
    self.c.execute("INSERT INTO query VALUES " + query)

    self.len_query = len(args)

  def execute_query(self, query):
    """
    Executes SQL query on DuckDB database.

    Args:
      query = [str] the SQL query to be executed by DuckDB

    Returns [DataFrame]:
      The output of the execution as a Pandas DataFrame object.
    """
    out = self.c.execute(query)
    return out.fetchdf()

  def _C(self):
    """ 
    Gets total number of terms in the index.
    
    Returns [int]:
      Total number of indexed terms.
    """
    C = self.c.execute("SELECT SUM(dict.cf) "
                       "FROM dict")
    return C.fetchdf().iloc[0, 0]

  def _N(self):
    """
    Gets number of documents in the index.

    Returns [int]:
      Number of indexed documents.
    """
    N = self.c.execute("SELECT COUNT(*) "
                       "FROM docs")
    return N.fetchdf().iloc[0, 0]

  def _avgdl(self):
    """
    Gets average number of terms per document in the index.

    Returns [float]:
      Average length of indexed documents.
    """
    avgdl = self.c.execute("SELECT AVG(docs.len) "
                           "FROM docs")
    return avgdl.fetchdf().iloc[0, 0]

     |████████████████████████████████| 1.3MB 10.2MB/s 
  Created wheel for duckdb: filename=duckdb-0.1.1-cp36-cp36m-linux_x86_64.whl size=1947901 sha256=aed9ade39cbbeb9318c40f860b8495b2aad53198495999e0da27e0fc787c8a65
  Stored in directory: /root/.cache/pip/wheels/31/2e/81/8061e62cf80a0ea73a4657d5807c46a93105440af5921e828c
Successfully built duckdb


In [4]:
!pip install monetdblite
import monetdblite as m
import pandas as pd

class MonetDBLite(object):
  """ 
  Class that houses all the MonetDBLite functionalities. 

  Attributes:
    C     = [int] number of indexed terms
    N     = [int] number of indexed documents
    avgdl = [float] average number of terms per document
    len_query = [int] number of terms in current search query
  """
  def __init__(self,
               database='/tmp/MonetDBLite Database',
               dict='/content/dict.csv',
               docs='/content/docs.csv',
               terms='/content/terms.csv'):
    """
    Initializes MonetDBLite database with index.

    Args:
      database = [str] database path
      dict     = [str] filename for dictionary CSV
      docs     = [str] filename for documents CSV
      terms    = [str] filename for terms CSV
    """
    m.init(database)

    # copy dictionary CSV into MonetDBLite database
    m.sql("CREATE TABLE dict(termid INTEGER "
                           ",term   VARCHAR(99) "
                           ",df     INTEGER "
                           ",cf     INTEGER)")
    m.sql("COPY INTO dict "
          "FROM '" + dict + "' "
          "USING DELIMITERS '|'")
    
    # copy documents CSV into MonetDBLite database
    m.sql("CREATE TABLE docs(name  VARCHAR(99) "
                           ",docid INTEGER "
                           ",len   INTEGER "
                           ",temp  INTEGER)") 
    m.sql("COPY INTO docs "
          "FROM '" + docs + "' "
          "USING DELIMITERS '|'")
    
    # copy terms CSV into MonetDBLite database
    m.sql("CREATE TABLE terms(termid INTEGER "
                            ",docid  INTEGER "
                            ",pos    INTEGER "
                            ",tf     INTEGER)")
    m.sql("COPY INTO terms "
          "FROM '" + terms + "' "
          "USING DELIMITERS '|'")
    
    # compute standard index statistics
    self.C = self._C()
    self.N = self._N()
    self.avgdl = self._avgdl()
    self.len_query = 0
    
  def make_query(self, *args: str):
    """
    Makes query table in MonetDBLite database filled with query terms.
    
    Args:
      args = [[str]] concatenation of strings to be made into a query
    """
    query = "('" + args[0] + "')"
    for arg in args[1:]:
        query += ", ('" + arg + "')"
    
    m.sql("DROP TABLE IF EXISTS query")
    m.sql("CREATE TABLE query(term VARCHAR(99))")
    m.sql("INSERT INTO query VALUES " + query)

    self.len_query = len(args)

  def execute_query(self, query):
    """
    Executes SQL query on MonetDBLite database.

    Args:
      query = [str] the SQL query to be executed by MonetDBLite

    Returns [DataFrame]:
      The output of the execution as a Pandas DataFrame object.
    """
    out = m.sql(query)
    return pd.DataFrame.from_dict(out)

  def _C(self):
    """ 
    Gets total number of terms in the index.
    
    Returns [int]:
      Total number of indexed terms.
    """
    C = m.sql("SELECT SUM(dict.cf) "
              "FROM dict")
    return pd.DataFrame.from_dict(C).iloc[0, 0]

  def _N(self):
    """
    Gets number of documents in the index.

    Returns [int]:
      Number of indexed documents.
    """
    N = m.sql("SELECT COUNT(*) "
              "FROM docs")
    return pd.DataFrame.from_dict(N).iloc[0, 0]

  def _avgdl(self):
    """
    Gets average number of terms per document in the index.

    Returns [float]:
      Average length of indexed documents.
    """
    avgdl = m.sql("SELECT AVG(docs.len) "
                  "FROM docs")
    return pd.DataFrame.from_dict(avgdl).iloc[0, 0]

     |████████████████████████████████| 7.3MB 8.1MB/s 


In [0]:
class Retriever(object):
  """ Class to do document retrieval with term proximity using databases. """     
  def retrieve(self,
               query, 
               db,
               con_query=True, 
               pre_select='kld', 
               tp=True,
               k=20,
               sum=True,
               mu=0.8, # totally not sure about this hyper-parameter
               k1=1.2,
               b=0.75,
               num_docs=100,
               max_span=5):
    """
    Function that retreives documents with a Retrieval Status Value (RSV)
    based on term-proximity (TP) weighting, Okapi BM25 or Kullback-Leibler
    Divergence. When opting for TP, k documents can be pre-selected with
    the Okapi BM25 or Kullback-Leibler Divergence retrieval models.

    Args:
      query      = [[str]] the tokenized and normalied query
      db         = [DuckDB|MonetDBLite] database that stores the index
      con_query  = [bool] whether all query terms need to be in
                          the document for it to be retrieved
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model
      tp         = [bool] whether to do the term proximity at all
      k          = [int] maximum number of documents to retrieve with
                         the pre-selection retrieval model
      sum        = [bool] whether to sum the pre-selection and term 
                          proximity scores for the final score
      mu         = [float] hyper-parameter for the KLD retrieval model
      k1         = [float] hyper-parameter for Okapi BM25
      b          = [float] hyper-parameter for Okapi BM25
      num_docs   = [int] maximum number of documents to retrieve
      max_span   = [int] maximum distance, in number of terms, for a term
                         pair to be included in the term proximity score

    Returns [DataFrame]:
      The Pandas DataFrame output.
    """
    db.make_query(*query)

    sql = (self._qterms(pre_select, tp) +
           self._qtermstf(pre_select, tp) +
           self._condocs(db, con_query, tp) +
           self._pre_select_subscores(db, con_query, pre_select, mu, k1, b) +
           self._topkdocs(pre_select, k) + 
           self._pairs(con_query, pre_select, tp, max_span) +
           self._tpscores(db, tp, k1, b) +
           self._scores(pre_select, tp, sum, num_docs))

    return db.execute_query(sql)

  def _qterms(self, pre_select, tp):
    """ 
    Get the SQL query that will retrieve the rows in the terms file
    belonging to the query terms, including the positional information.

    Args:
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model;
                   Kullback-Leibler Divergence retrieval model also needs
                   collection frequency information of each term  
      tp         = [bool] whether to do the term proximity

    Returns [str]:
      SQL query as string.
    """
    query = ("WITH qtermids "
                  "AS (SELECT dict.termid "
                            ",dict.df "
                            "{}"
                      "FROM dict "
                      "JOIN query "
                      "ON dict.term = query.term"
                      ") "
             "{}")
    
    if tp:
      query = query.format("{}",
                           ", qterms "
                           "AS (SELECT terms.termid "
                                     ",terms.docid "
                                     ",terms.pos "
                                     ",terms.tf "
                                     ",qtermids.df "
                                     "{}"
                               "FROM terms "
                               "JOIN qtermids "
                               "ON terms.termid = qtermids.termid"
                               ") ")
      if pre_select == 'kld':
        return query.format(",dict.cf ", ",qtermids.cf ")
      else:
        return query.format("", "")
    else:
      if pre_select == 'kld':
        return query.format(",dict.cf ", "")
      else:
        return query.format("", "")  

  def _qtermstf(self, pre_select, tp):
    """
    Get the SQL query that will retrieve the rows in the terms file
    belonging to the query terms, excluding the positional information.

    Args:
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model;
                   Kullback-Leibler Divergence retrieval model also needs
                   collection frequency information of each term
      tp         = [bool] whether to do the term proximity

    Returns [str]:
      SQL query as string.
    """
    if pre_select == 'none':
      return ""
    
    query = (", qtermstfrows "
                  "AS (SELECT qterms.termid "
                            ",qterms.docid "
                            ",qterms.tf "
                            "{}"
                            "{}"
                            ",( ROW_NUMBER() "
                               "OVER(PARTITION BY qterms.termid, qterms.docid "
                                    "ORDER BY qterms.pos"
                                    ")"
                              ") AS row "
                      "{}" 
                      ") "
             ", qtermstf "
                  "AS (SELECT qtermstfrows.termid "
                            ",qtermstfrows.docid "
                            ",qtermstfrows.tf "
                            ",qtermstfrows.df "
                            "{}"
                      "FROM qtermstfrows "
                      "WHERE qtermstfrows.row = 1"
                      ") ")

    if pre_select == 'kld':
      if tp:
        return query.format(",qterms.df ",
                            ",qterms.cf ",
                            "FROM qterms",
                            ",qtermstfrows.cf ")
      else:
        return query.format(",qtermids.df ",
                            ",qtermids.cf ",
                            "FROM terms AS qterms "
                            "JOIN qtermids "
                            "ON qterms.termid = qtermids.termid",
                            ",qtermstfrows.cf ")
    elif pre_select == 'okapi':
      if tp:
        return query.format(",qterms.df ",
                            "",
                            "FROM qterms",
                            "")
      else:
        return query.format(",qtermids.df ",
                            "",
                            "FROM terms AS qterms "
                            "JOIN qtermids "
                            "ON qterms.termid = qtermids.termid",
                            "")

  def _condocs(self, db, con_query, tp):
    """
    Get the SQL query that will retrieve the rows in the terms file
    belonging to documents that contain all the query terms.

    Args:
      db        = [DuckDB|MonetDBLite] database that stores the index
      con_query = [bool] whether all query terms need to be in
                         the document for it to be retrieved
      tp         = [bool] whether to do the term proximity

    Returns [str]:
      SQL query as string.                         
    """
    if con_query:
      query = (", condocs "
                    "AS (SELECT qterms.docid "
                        "FROM {} AS qterms "
                        "GROUP BY qterms.docid "
                        "HAVING COUNT(DISTINCT qterms.termid) = {:d}"
                        ") ")
      if tp:
        return query.format("qterms", db.len_query)
      else:
        return query.format("qtermstf", db.len_query)
    else:
      return ""

  def _pre_select_subscores(self, db, con_query, pre_select, mu, k1, b):
    """
    Get the SQL query that will compute a score for each 
    query term-document pair, according to the pre-selection
    retrieval model.

    Args:
      db         = [DuckDB|MonetDBLite] database that stores the index
      con_query  = [bool] whether all query terms need to be in
                          the document for it to be retrieved     
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model
      mu         = [float] hyper-parameter for the KLD retrieval model
      k1         = [float] hyper-parameter for Okapi BM25
      b          = [float] hyper-parameter for Okapi BM25
    
    Returns [str]:
      SQL query as string.
    """
    if pre_select == 'kld':
      query = (", kldsubscores "
                    "AS (SELECT qtermstf.docid "
                              ",( LOG({:f}+tf*{:f}/cf)"              
                                  "+" 
                                 "LOG(1/({:f}+len))"
                                ") AS subscore "
                        "FROM qtermstf "
                        "{}"
                        "JOIN docs "
                        "ON qtermstf.docid = docs.docid"
                        ") ")
      query = query.format(mu, db.C, mu, "{}")
    elif pre_select == 'okapi':
      query = (", okapisubscores "
                    "AS (SELECT qtermstf.docid "
                              ",( LOG(({:f}-df+0.5)/(df+0.5))*tf*({:f}+1)"
                                  "/"
                                 "(tf+{:f}*(1-{:f}+{:f}*len/{:f}))"
                                ") AS subscore "
                        "FROM qtermstf "
                        "{}"
                        "JOIN docs "
                        "ON qtermstf.docid = docs.docid"
                        ") ")
      query = query.format(db.N, k1, k1, b, b, db.avgdl, "{}")
    else:
      return ""

    if con_query:
      return query.format("JOIN condocs "
                          "ON qtermstf.docid = condocs.docid ")
    else:
      return query.format("")

  def _topkdocs(self, pre_select, k):
    """
    Get the SQL query that will compute the pre-selection scores,
    according to the pre-selection retrieval model, and retrieve
    the top k documents.

    Args:
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model
      k          = [int] maximum number of documents retrieved with
                       the pre-selection retrieval model
    Returns [str]:
      SQL query as string.
    """
    if pre_select == 'none':
      return ""

    query = (", topdocs "
                  "AS (SELECT subscores.docid "
                            ",SUM(subscores.subscore) AS score "
                            ",( ROW_NUMBER() "
                                "OVER(ORDER BY SUM(subscores.subscore) DESC)"
                              ") AS row "
                      "FROM {} AS subscores "
                      "GROUP BY subscores.docid"
                      ") "
              ", topkdocs "
                  "AS (SELECT topdocs.docid "
                            ",topdocs.score "
                      "FROM topdocs "
                      "WHERE topdocs.row BETWEEN 1 AND {:d}"
                      ") ")
    
    if pre_select == 'kld':
      return query.format("kldsubscores", k)
    elif pre_select == 'okapi':
      return query.format("okapisubscores", k)

  def _pairs(self, con_query, pre_select, tp, max_span):
    """
    Get the SQL query that will compute the term pair instance (tpi) for
    each query term pair within a span of max_span terms.

    Args:
      con_query  = [bool] whether all query terms need to be in
                          the document for it to be retrieved                          
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model  
      tp         = [bool] whether to do the term proximity                   
      max_span   = [int] the maximum span, in terms, of a term pair to
                        include in the term proximity score

    Returns [str]:
      SQL query as string.
    """
    if not tp:
      return ""

    query = (", pairs "
                  "AS (SELECT qterms1.termid AS termid1 "
                            ",qterms2.termid AS termid2 "
                            ",qterms1.docid "
                            ",1.0/(qterms1.pos-qterms2.pos) AS tpi "
                            ",( CASE WHEN qterms1.df > qterms2.df THEN qterms1.df "
                                    "ELSE qterms2.df "
                               "END"
                              ") AS maxdf "
                      "FROM qterms AS qterms1 "
                      "{}"
                      "{}"
                      "JOIN qterms AS qterms2 "
                      "ON qterms1.docid = qterms2.docid AND "
                         "NOT qterms1.termid = qterms2.termid AND "
                         "qterms1.pos - qterms2.pos BETWEEN 1 AND {:d}"
                      ") ")
    if con_query:
      query = query.format("JOIN condocs ON qterms1.docid = condocs.docid ",
                           "{}",
                           max_span)
    else:
      query = query.format("",
                           "{}",
                           max_span)
      
    if pre_select == 'none':
      return query.format("")
    else:
      return query.format("JOIN topkdocs ON qterms1.docid = topkdocs.docid ")
      
  def _tpscores(self, db, tp, k1, b):
    """
    Get the SQL query that will compute the term proximity score.

    Args:      
      db        = [DuckDB|MonetDBLite] database that stores the index
      tp       = [bool] whether to do the term proximity
      k1       = [float] hyper-parameter for Okapi BM25
      b        = [float] hyper-parameter for Okapi BM25

    Returns [str]:
      SQL query as string.
    """
    if not tp:
      return ""
      
    query = (",tpisums "
                  "AS (SELECT pairs.termid1 "
                            ",pairs.termid2 "
                            ",pairs.docid "
                            ",SUM(pairs.tpi) AS tpisum "
                      "FROM pairs "              
                      "GROUP BY pairs.termid1 "
                              ",pairs.termid2 "
                              ",pairs.docid"
                      ") "
            ", tpsubscores "
                  "AS (SELECT pairs.docid "
                            ",( LOG(({:f}-maxdf+0.5)/(maxdf+0.5))*tpisum*({:f}+1)"
                                "/"
                              "(tpisum+{:f}*(1-{:f}+{:f}*len/{:f}))"
                              ") AS tpsubscore "
                      "FROM pairs "
                      "JOIN tpisums "
                      "ON pairs.termid1 = tpisums.termid1 AND "
                        "pairs.termid2 = tpisums.termid2 AND "
                        "pairs.docid = tpisums.docid "
                      "JOIN docs "
                      "ON pairs.docid = docs.docid"
                      ") "
            ", tpscores "
                  "AS (SELECT tpsubscores.docid "
                            ",SUM(tpsubscores.tpsubscore) AS tpscore "
                      "FROM tpsubscores "
                      "GROUP BY tpsubscores.docid"
                      ") ")
    return query.format(db.N, k1, k1, b, b, db.avgdl)

  def _scores(self, pre_select, tp, sum, num_docs):
    """ 
    Get the SQL query that will retrieve or compute the final document scores.

    Args:      
      pre_select = ['kld'|'okapi'|'none'] pre-selection retrieval model 
      tp         = [bool] whether to do the term proximity
      sum        = [bool] whether to sum the pre-select and term 
                          proximity scores for the final score
      num_docs   = [int] maximum number of documents to retrieve

    Returns [str]:
      SQL query as string.
    """
    query = (", scores "
                  "AS ({}) "
             "SELECT scores.docid "
                   ",scores.score "
             "FROM scores "
             "ORDER BY scores.score DESC "
             "LIMIT {:d}")
    
    if not tp:
      return query.format("SELECT topkdocs.docid "
                                ",topkdocs.score "
                          "FROM topkdocs",
                          num_docs)
    elif pre_select == 'none' or not sum:
      return query.format("SELECT tpscores.docid "
                                ",tpscores.tpscore AS score "
                          "FROM tpscores",
                          num_docs)
    else:
      return query.format("SELECT topkdocs.docid "
                                ",( topkdocs.score"
                                    "+"
                                   "COALESCE(tpscores.tpscore, 0)"
                                  ") AS score "
                          "FROM topkdocs "
                          "LEFT JOIN tpscores "
                          "ON topkdocs.docid = tpscores.docid",
                          num_docs)

In [0]:
duck = DuckDB()
monet = MonetDBLite()

In [0]:
retriever = Retriever()

In [20]:
query = ['new', 'york']

duck_time = datetime.now()
duck_scores = retriever.retrieve(query, duck)
duck_time = datetime.now() - duck_time
print("DuckDB query time: {}".format(duck_time))

monet_time = datetime.now()
monet_scores = retriever.retrieve(query, monet)
monet_time = datetime.now() - monet_time
print("MonetDBLite query time: {}".format(monet_time))

DuckDB query time: 0:00:13.023675
MonetDBLite query time: 0:00:00.375717


In [21]:
duck_scores

,docid,score
0,50961,59.406660
1,100831,18.508918
2,330675,13.951959
3,254096,11.489001
4,323541,10.929049
5,187978,8.859076
6,60420,7.729182
7,256566,6.604098
8,134955,5.923230
9,111825,5.485813


In [22]:
monet_scores

,docid,score
0,50961,136.788891
1,100831,42.618359
2,330675,32.125574
3,254096,26.454403
4,323541,25.165066
5,187978,20.398776
6,60420,17.797100
7,256566,15.206498
8,134955,13.638742
9,111825,12.631551
